In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## Refs.

1. Introduction to the theory of neuronal computation, Hertz, Krogh, Palmer (1991)

2. https://machinelearningmastery.com/implement-perceptron-algorithm-scratch-python/

3. https://towardsdatascience.com/perceptron-explanation-implementation-and-a-visual-example-3c8e76b4e2d1

## Teoría

### Múltiples neuronas de salida

Consideramos un perceptrón simple (una capa) de $n$ entradas y $m$ neuronas de salida.
El vector $x\in\mathbb{R}^n$ representa el estado de las neuronas de entrada, y el vector $y\in\mathbb{R}^m$ el estado de las neuronas de salida.
El estado de la $j$-ésima neurona de salida viene determinado por

$$ g(b_j+w_jx) \;\;\; (1)$$

donde la constante $b_j\in \mathbb{R}$ se llama umbral, y el vector $w_j\in \mathbb{R}^n$ representa pesos sinápticos.
Más precisamente, $b$ es un vector de umbrales y $w\in\mathbb{R}^{m,n}$ es una matriz de pesos sinápticos, donde

$$ w_jx = \Sigma_{i=1}^n w_{ji}x_i $$ 

es un producto escalar, $w_j$ es la $j$-esima fila de $w$, y $g\in (\mathbb{R} \to \mathbb{R})$ es alguna función activación (i.e. una función no decreciente que poseen alguna no linealidad).
Notar que todas las neuronas de salida usan la misma función activación $g$.
Existen muchas funciones de activación, pero nos enfocaremos en usar

\begin{eqnarray}
g(x)
&:=& \tanh(\beta x) \\
&=& \frac{e^{\beta x}-e^{-\beta x}}{e^{\beta x}+e^{-\beta x}} \\
\end{eqnarray}

la cual graficamos a continuación para diferentes valores de $\beta$

In [ ]:
beta = 1.0
def g(x):
    return np.tanh(beta*x)

x = np.linspace(-3,3,1000)
for _beta in [0.1,1.0,10.0,100.0]:
    beta = _beta
    plt.plot(x,np.vectorize(g)(x),label='beta='+str(beta))
    
plt.xlabel("x")
plt.ylabel("$g_{\\beta}$")
plt.legend()

Esta $g$ es diferenciable en $\mathbb{R}$ y satisface

$$ g' = \beta(1-g^2) $$

lo cual, convenientemente, puede ser usado para ahorrar recursos de cómputo.
A continuación graficamos ésta derivada

In [ ]:
def dg(x):
    return beta*(1.0-g(x)**2)

x = np.linspace(-3,3,1000)
for _beta in [2**i for i in range(-2,3)]:
    beta = _beta
    plt.plot(x,np.vectorize(lambda x:dg(x)/beta)(x),label='beta='+str(beta))
    
plt.xlabel("x")
plt.ylabel("$g^{'}/\\beta$")
plt.legend()

en donde vemos que se transforma en un pico cuyo máximo crece como $\sim \beta$ en torno a $x=0$.

Un **truco** útil consiste en notar que 

$$ b_j + w_jx = b_j + \sum_{i=0}^n w_{ji}x_i $$ 

puede remplazarse por 

$$ w_jx = w_{j0}x_0 + \sum_{i=1}^n w_{ji}x_i = \sum_{i=0}^n w_{ji}x_i $$ 

si introducimos los pesos sinápticos $w_{j0}=b_j$ y una $0$-ésima neurona artificial de entrada de estado permanente $x_0=1$.

#### Entrenamiento

Consideremos una serie de datos de entrada $x_1,...,x_q$ donde $x_k\in \mathbb{R}^{1+n}$ para todo $k=1,...,q$, y una serie de datos de salida $y_1,...,y_q$ donde $y_k\in \mathbb{R}^m$ para todo $k=1,...,q$.
Estos datos podrían ser generados por experimentos o sintéticamente, recordando que hay que fijar $x_{k0}=1$ para todo $k$ para poder aprovechar el **truquito** anteriormente mencionado.

El objetivo es entrenar los pesos sinápticos de la red hasta que ésta logre aproximar, de la mejor manera posible, la relación entre los datos de entrada y los de salida.
Formalmente, buscamos minimizar la suma de errores cuadráticos

\begin{eqnarray}
e&:=&\sum_{k=1}^q \sum_{j=1}^m (y_{kj}-g(w_jx_k))^2 \\
&=&\sum_{k=1}^q \sum_{j=1}^m (y_{kj}-g(z_{kj}))^2 \\
&=&\sum_{k=1}^q \sum_{j=1}^m (y_{kj}-v_{kj})^2 \\
&=&\sum_{k=1}^q (y_k-v_k)^2 \\
\end{eqnarray}

con respecto a $w$, donde, en la última linea, $(y_k-v_k)^2$ representa el producto escalar del vector $y_k-v_k$ consigo mismo.
Además, reconocemos aquí que los datos de salida constituyen una matriz $y\in \mathbb{R}^{q,m}$.
Además, hemos introducido el $k$-ésimo vector de salida $v_k\in \mathbb{R}^m$, el cuál constituye la predicción que realiza la red neuronal cuando se la expone a la $k$-ésima entrada $x_k$, cuya $j$-ésima componente es

$$ v_{kj} := g(z_{kj}) $$

donde

\begin{eqnarray}
z^T_{jk} & := & w_jx_k \\
&=& \sum_{i=0}^n w_{ji}x^T_{ik} \\
&=& (wx^T)_{jk} \\
\end{eqnarray}

Si pensamos a $e$ como una función $e\in (\mathbb{R}^{m,(1+n)}\ni w\to \mathbb{R})$, podemos intentar minimizarla descendiendo por el gradiente de componentes

\begin{eqnarray}
\frac{\partial e}{\partial w_{ji}}
&=& 2\sum_{k=1}^q \sum_{s=1}^m (y_{ks}-g(z_{ks}))\frac{\partial}{\partial w_{ji}}(y_{ks}-g(z_{ks})) \\
&=& -2\sum_{k=1}^q \sum_{s=1}^m (y_{ks}-g(z_{ks}))g'(z_{ks})\frac{\partial}{\partial w_{ji}}z_{ks} \\
&=& -2\sum_{k=1}^q \sum_{s=1}^m (y_{ks}-g(z_{ks}))g'(z_{ks})\frac{\partial}{\partial w_{ji}} \sum_{r=0}^n w_{sr}x_{kr} \\
&=& 2\sum_{k=1}^q \sum_{s=1}^m (g(z_{ks})-y_{ks})g'(z_{ks}) \sum_{r=0}^n x_{kr} \delta_{ir} \delta_{js} \\
&=& 2\sum_{k=1}^q \sum_{s=1}^m (g(z_{ks})-y_{ks})g'(z_{ks}) x_{ki} \delta_{js} \\
&=& 2\sum_{k=1}^q (g(z_{kj})-y_{kj})g'(z_{kj}) x_{ki}  \\
&=& 2\sum_{k=1}^q (v_{kj}-y_{kj})u_{kj} x_{ki} \\
\end{eqnarray}

donde $x_{ki}$ es la $i$-ésima componente de la $k$-ésima muestra de entrada $x_k$.
Además, por cuestiones prácticas, en la última línea hemos introducido el $k$-ésimo vector $u_k\in \mathbb{R}^m$ de $j$-ésima componente

$$ u_{kj} := g'(z_{kj}) $$

Para minimizar $e$ con respecto a $w$, consideramos un algoritmo iterativo en donde $w^{(0)},w^{(1)},...,w^{(t)},w^{(t+1)},...$ denotan los valores de $w$ en cada paso de iteración.
Un sencillo algoritmo de minimización por el gradiente consiste en inicializar $w^{(0)}$ con valores elegidos al azar de alguna distribución centrada en 0, para luego ir actualizandolos iterativamente según las regla

$$ w^{(t+1)}_{ji} := w^{(t)}_{ji} - r \left.\frac{\partial e}{\partial w_{ji}}\right|_{w^{(t)}} $$

donde $r>0$ es algún valor pequeño que determina la tasa de convergencia.
La regla debe aplicarse para todo $i$ en cada iteración, hasta que veamos que el error $e$ deja de decrecer, estacionandose en algún valor.

#### Minipráctico

**a)** Usar `scikit-learn.datasets.make_classification` para crear un dataset para clasificación con:

- 2 características (features)
- 2 clases
- 200 muestras
- sin redundancia
- 1 grupo (cluster) por clase

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
n = 2
m = 2
q = 100 # k=0,1,...,q-1 donde q = número de muestras.

num_classes = 2**m # hipercubo de 2^m vértices, o número binario de m dígitos.

x_ki,y_k = make_classification(
    n_features=n,
    n_classes=num_classes,
    n_samples=q,
    n_redundant=0,
    n_clusters_per_class=1
)

In [ ]:
x_ki.shape,y_k.shape

In [ ]:
#x_ki

In [ ]:
y_k

In [ ]:
# Desdoblamos el índice y_k del 2^m clusters en la señal binaria de m neuronas de salida.
# Más precisamente, hacemos
#   cluster | número binario
#       y_k | y_kj = b_2 b_1
#         0 |          0   0
#         1 |          0   1
#         2 |          1   0
#         3 |          1   1
# donde b_j es el j-ésimo dígito menos significativo del numero binario b = b_2 b_1.

def binario_de_m_digitos(y,m):
    """
    Entrada 
        i : entero
    Salida
        b : binario de m digitos correspondiente a entero i.
    """
    return [float(n) for n in bin(y)[2:].zfill(m)]

y_kj = np.zeros((q,m))
for k in range(q):
    y_kj[k,:] = binario_de_m_digitos(y_k[k],m)
#y_kj

**b)** Grafique el dataset generado.

In [ ]:
color = {0:'red',1:'blue',2:'green',3:'cyan'}
for k in range(q):
    plt.scatter([x_ki[k,0]],[x_ki[k,1]],c=color[y_k[k]])
plt.title("datos")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")

**c)** Implemente un perceptron simple de $2$ neuronas de entrada y una de salida, utilizando

$$ g(x) = \tanh(\beta x) $$ 

como función activación.

In [ ]:
# Agregamos la columna truquito
tmp = np.ones((q,1+n))
tmp[:,1:1+n] = x_ki
x_ki = tmp
#x_ki

In [ ]:
def perceptron(w_ji,x_i):
    return np.vectorize(g)(np.dot(w_ji,x_i))

**d)** Implemente una funcion que entrene el perceptron. Utilice

- $\beta=1$
- $r=0.01$
- y $t_{max}=100$ iteraciones

El entrenador debe tomar como entrada, entre otras cosas,

- el dataset de entrenamiento, i.e. los vectores $x$,$y$, de componentes $x_{ki}$ e $y_k$ correspondiendo a la muestra $k$-ésima y la neurona de entrada $i$-ésima.
- el vector de pesos sinápticos $w$

y debe retornar un vector de componentes $e_t =$ que indiquen el error cuadrático tras $t$ iteraciones.
Durante el proceso de entrenamiento, el entrenador debe modificar el vector $w$ que es pasado como argumento.

Grafique $e_t$ vs $t$.

In [ ]:
def entrenar(w_ji,x_ki,y_k,r,tmax,g):
    for t in range(tmax):
        z_kj = np.dot(w_ji,x_ki.T).T
        v_kj = np.vectorize(g)(z_kj)
        u_kj = beta*(1-v_kj**2)
        e_kj = (v_kj-y_kj)
        eu_kj = e_kj*u_kj
        w_ji -= 2*r*np.dot(x_ki.T,eu_kj).T
        #e = np.linalg.norm(e_kj)        
        e = np.dot(e_kj.flatten(),e_kj.flatten())        
        yield t,e

In [ ]:
# Inicializamos pesos sinápticos
w_ji = np.random.normal(size=(m,1+n))
w_ji

In [ ]:
beta=1.0
r=0.01
t_max=500

tvec,evec=[],[]
for t,e in entrenar(w_ji,x_ki,y_kj,r,t_max,g):
    tvec.append(t)
    evec.append(e)
plt.plot(tvec,evec)
plt.xlabel("t")
plt.ylabel("e")

**e)** Grafique el resultado del entrenamiento

In [ ]:
def float_a_binary(x):
    if x>0.5:
        return 1
    return 0

def salida_a_integer(y):
    b = np.vectorize(float_a_binary)(y)
    return int('0b'+''.join([str(d) for d in b]),2)

# Tiene que retornar el entero asociado a 011 que es 3.
salida_a_integer([0.1,0.9,0.8])

In [ ]:
# Ploteamos el resultado del entrenamiento
for k in range(q):
    c = color[salida_a_integer(perceptron(w_ji,x_ki[k,:]))]
    plt.scatter([x_ki[k,1]],[x_ki[k,2]],c=c)

# Agregamos linea de separacion
xmin = np.min(x_ki[:,1])
xmax = np.max(x_ki[:,1])
xs = np.linspace(xmin,xmax,100)
ymax = np.max(x_ki[:,2])
ymin = np.min(x_ki[:,2])
plt.title("prediccion")

**f)** Repita todo lo anterior probando con otras funciones de activación, valores de $r$ y de $\beta$.

**g)** Repita todo lo anterior usando un mapeo diferente entre las clases generadas con `scikit-learn` y la señal de salida a aprender. 
Más precisamente, el mapeo por el cual $y_{kj}=\delta_{jc}$ si la $k$-ésima entrada $x_k$ corresponde a la clase $c$.
Aquí, $\delta_{jc}=1$ si $j=c$ y $\delta_{jc}=0$ en caso contrario (esta delta se llama, la delta de Kronecker).